In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [2]:
df_pars_train = pd.read_csv('Trainset_features_10000.csv')
df_pars_test = pd.read_csv('Testset_features.csv')

pars_train = np.array(df_pars_train)
pars_test = np.array(df_pars_test)

X = pars_train[:,3:]
y = pars_train[:,2]

X_testset = pars_test[:,2:]

Each feature accuracy
3. Out degree source
4. In degree source
5. Out degree destination
6. In degree destination
7. Transitive friends
8. Outdegree Common friends
9. Indegree common friends
10. Total friends
11. Jaccard
12. Cosine

In [10]:
selected_columns = [3, 4, 5, 6, 13]

X = pars_train[:,selected_columns]
y = pars_train[:,2]
print(X)

[[2.150e+02 2.100e+01 0.000e+00 1.000e+00 2.360e+02]
 [3.010e+02 4.900e+01 0.000e+00 1.000e+00 3.500e+02]
 [7.940e+02 3.600e+01 0.000e+00 5.000e+00 4.150e+03]
 ...
 [3.800e+02 1.710e+02 0.000e+00 1.100e+01 6.061e+03]
 [4.300e+01 2.600e+01 0.000e+00 1.000e+00 6.900e+01]
 [5.660e+02 4.620e+02 0.000e+00 5.500e+01 5.654e+04]]


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

RANDOM FOREST

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score

# sc = StandardScaler()
# sc.fit(X_train)
# X_train_std = sc.transform(X_train)
# X_test_std = sc.transform(X_test)
# X_testset_std=sc.transform(X_testset)

classifier = RandomForestRegressor(n_estimators = 1000)
classifier.fit(X_train, y_train)

y_pred_RF = classifier.predict(X_test)
y_pred_RF = np.where(y_pred_RF > 0.5, 1, 0)
acc_RF =accuracy_score(y_test, y_pred_RF)

In [13]:
acc_RF

0.75325

In [ ]:
y_pred_RF_test = classifier.predict(X_testset_std)
df_pred_RF_test = pd.DataFrame(y_pred_RF_test)
df_pred_RF_test

XGBOOST

In [14]:
import xgboost as xgb

imbalance_ratio = (y_train == 0).sum() / (y_train == 1).sum()
sample_weights = [2.0] * len(X_train)
classifier_XGB = xgb.XGBClassifier(eta=0.01, scale_pos_weight=imbalance_ratio)
classifier_XGB.fit(X_train, y_train)

y_pred_XGB = classifier_XGB.predict(X_test)
acc_XGB =accuracy_score(y_test, y_pred_XGB)
acc_XGB

0.791

In [ ]:
#y_pred_XGB = classifier_XGB.predict(X_testset)
y_prob_XGB = classifier_XGB.predict_proba(X_testset_std)
y_prob_class_1_XGB = y_prob_XGB[:, 1]
#df_y_prob_XGB = pd.DataFrame(y_prob_class_1_XGB)
#df_y_pred_XGB = pd.DataFrame(y_pred_XGB)
y_prob_class_1_XGB
# df_y_pred_XGB.to_csv('Test_submission_XGB_scaled_20000.csv')

In [ ]:
df_y_prob_XGB = pd.DataFrame(y_prob_class_1_XGB)
df_y_prob_XGB

In [ ]:
#df_y_prob_XGB.to_csv('Test_submission_XGB_20000_0110.csv')

ADABOOST

In [16]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics

In [17]:
abc = AdaBoostClassifier(n_estimators=50, learning_rate=1)
# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred_AB = model.predict(X_test)
print("Accuracy score %.3f" %metrics.accuracy_score(y_test, y_pred_AB))

Accuracy score 0.791


In [ ]:
#y_pred_AB =abc.predict(X_testset)
y_prob_AB = model.predict_proba(X_testset_std)
y_prob_class_1_AB = y_prob_AB[:, 1]
df_y_pred_AB = pd.DataFrame(y_prob_class_1_AB)
df_y_pred_AB

SVM

In [18]:
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn import metrics

In [19]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std_SVM = sc.transform(X_train)
X_test_std_SVM = sc.transform(X_test)
X_testset_std_SVM=sc.transform(X_testset)

ValueError: X has 11 features, but StandardScaler is expecting 5 features as input.

In [20]:
# Instantiate the Support Vector Classifier (SVC)
svc = SVC(C=10, random_state=1, kernel='rbf', probability=True)
 
# Fit the model
svc.fit(X_train, y_train)

SVC(C=10, probability=True, random_state=1)

In [22]:
y_pred_SVM = svc.predict(X_test)
 
# Measure the performance
print("Accuracy score %.3f" %metrics.accuracy_score(y_test, y_pred_SVM))

Accuracy score 0.664


In [ ]:
#y_pred_SVM = svc.predict(X_testset_std_SVM)
y_prob_SVM = svc.predict_proba(X_testset_std_SVM)
y_prob_class_1_SVM = y_prob_SVM[:, 1]
df_y_pred_SVM = pd.DataFrame(y_prob_class_1_SVM)
df_y_pred_SVM

KNN

In [23]:
from sklearn.neighbors import KNeighborsRegressor
knn_model = KNeighborsRegressor(n_neighbors=150)

from sklearn.metrics import mean_squared_error
from math import sqrt

In [25]:
knn_model.fit(X_train, y_train)
y_pred_KNN = knn_model.predict(X_test)
#mse = mean_squared_error(y_train, y_pred_KNN)
train_preds1 = np.where(y_pred_KNN > 0.5, 1, 0)
acc_KNN =accuracy_score(y_test,train_preds1)
acc_KNN

0.78775

In [ ]:
y_pred_KNN = knn_model.predict(X_testset)
df_y_pred_KNN = pd.DataFrame(y_pred_KNN)
#df_y_pred_KNN.to_csv('Test_submission_LR_20000.csv', index=False)
df_y_pred_KNN

In [ ]:
zeros = np.zeros(2)
zeros

NAIVE BAYES

In [26]:
from sklearn.naive_bayes import BernoulliNB
NaiveBayes_clf = BernoulliNB()
NaiveBayes_clf.fit(X_train, y_train)
y_pred_NB = NaiveBayes_clf.predict(X_test)
accuracy_score(y_test, y_pred_NB)

0.63875

In [ ]:
#y_pred_NB = NaiveBayes_clf.predict(X_testset)
log_probabilities = NaiveBayes_clf.predict_log_proba(X_testset)
probabilities = np.exp(log_probabilities)
y_prob_class_1_NB = probabilities[:, 1]
df_y_prob_NB = pd.DataFrame(y_prob_class_1_NB)
df_y_prob_NB

Logistic Reg

In [28]:
from sklearn.linear_model import LogisticRegression
model_LR = LogisticRegression(solver= 'liblinear', random_state=0)
model_LR.fit(X_train, y_train)
y_predict_LR = model_LR.predict(X_test)


 
# Measure the performance
print("Accuracy score %.3f" %metrics.accuracy_score(y_test, y_predict_LR))

Accuracy score 0.707


In [ ]:
#y_pred_LR = model_LR.predict(X_testset_std)
y_prob_LR = model_LR.predict_proba(X_testset_std)
y_prob_class_1_LR = y_prob_LR[:, 1]
df_y_pred_LR = pd.DataFrame(y_prob_class_1_LR)
df_y_pred_LR

ANN

In [29]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

# sc = StandardScaler()
# sc.fit(X_train)
# X_train_std = sc.transform(X_train)
# X_test_std = sc.transform(X_test)
# X_testset_std=sc.transform(X_testset)

# ann = tf.keras.Sequential()
# ann.add(tf.keras.layers.Dense(units=10,activation='relu'))
# ann.add(tf.keras.layers.Dense(units=64,activation='relu'))
# ann.add(tf.keras.layers.Dense(units=32,activation='relu'))
# ann.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))
# ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

ann = tf.keras.Sequential()

# Add the first hidden layer with Leaky ReLU activation
ann.add(tf.keras.layers.Dense(units=11, activation=tf.nn.leaky_relu))

# Add the second hidden layer with PReLU activation
ann.add(tf.keras.layers.Dense(units=128))
ann.add(tf.keras.layers.PReLU())

# Add the third hidden layer with PReLU activation
ann.add(tf.keras.layers.Dense(units=32))
ann.add(tf.keras.layers.PReLU())

# Add the output layer with sigmoid activation
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])




ann.fit(X_train,y_train,batch_size=32,epochs = 30 )
y_pred_ANN = ann.predict(X_test)
y_pred_ANN = np.where(y_pred_ANN > 0.5, 1, 0)
accuracy_score(y_test, y_pred_ANN)

Epoch 1/30
500/500 [==============================] - 1s 901us/step - loss: 1102.1115 - accuracy: 0.7013
Epoch 2/30
500/500 [==============================] - 0s 867us/step - loss: 726.3911 - accuracy: 0.7114
Epoch 3/30
500/500 [==============================] - 0s 865us/step - loss: 1053.8536 - accuracy: 0.7299
Epoch 4/30
500/500 [==============================] - 0s 864us/step - loss: 327.7672 - accuracy: 0.7511
Epoch 5/30
500/500 [==============================] - 0s 856us/step - loss: 376.3946 - accuracy: 0.7422
Epoch 6/30
500/500 [==============================] - 0s 877us/step - loss: 579.5347 - accuracy: 0.7134
Epoch 7/30
500/500 [==============================] - 0s 860us/step - loss: 561.9406 - accuracy: 0.7517
Epoch 8/30
500/500 [==============================] - 0s 872us/step - loss: 594.0258 - accuracy: 0.7488
Epoch 9/30
500/500 [==============================] - 0s 865us/step - loss: 232.8938 - accuracy: 0.7468
Epoch 10/30
500/500 [==============================] - 0s 870u

0.787

In [ ]:
y_pred_ANN =ann.predict(X_testset_std)
df_y_pred_ANN = pd.DataFrame(y_pred_ANN)
#df_y_pred_ANN.to_csv('Test_Submission_20000_ANN_0110_3.csv')

In [ ]:
ensemble_probs = np.column_stack((y_pred_ANN, y_prob_class_1_LR, y_prob_class_1_NB, y_pred_KNN, y_prob_class_1_SVM, y_prob_class_1_AB, y_prob_class_1_XGB,y_pred_RF_test))
average_probs = np.mean(ensemble_probs, axis=1)
df_y_pred_X_testset_sub = pd.DataFrame(average_probs)
#df_y_pred_X_testset_sub.to_csv('Test_submission_Ensemble_20000.csv')

In [ ]:
median_probs = np.median(ensemble_probs, axis=1)
df_y_pred_X_testset_sub = pd.DataFrame(median_probs)
#df_y_pred_X_testset_sub.to_csv('Test_submission_Ensemble_Median_20000.csv')

In [ ]:
#pip install tensorflow scikeras scikit-learn

In [ ]:
def create_ann_model():
    model = Sequential()
    # Add your layers here
    model.add(Dense(units=10, activation=tf.nn.leaky_relu, input_dim=X_train.shape[1]))
    model.add(Dense(units=128, activation=tf.nn.leaky_relu))
    model.add(Dense(units=32, activation=tf.nn.leaky_relu))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
ann_model = KerasClassifier(build_fn=create_ann_model, epochs=10, batch_size=32)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

def create_ann_model():
    model = Sequential()
    # Add your layers here
    model.add(Dense(units=11, activation=tf.nn.leaky_relu, input_dim=X_train.shape[1]))
    model.add(Dense(units=128, activation=tf.nn.leaky_relu))
    model.add(Dense(units=32, activation=tf.nn.leaky_relu))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Initialize each individual model (including the Keras model wrapped in KerasClassifier)
ann_model = KerasClassifier(build_fn=create_ann_model, epochs=20, batch_size=32)
svm_model = SVC(C=10, random_state=1, kernel='rbf', probability=True)
rf_model = RandomForestClassifier(n_estimators = 1000)
lr_model = LogisticRegression(solver= 'liblinear', random_state=0)
xgb_model = XGBClassifier(eta=0.01, scale_pos_weight=imbalance_ratio)
ada_model = AdaBoostClassifier(n_estimators=50, learning_rate=1)
nb_model = BernoulliNB()
knn_model = KNeighborsClassifier(n_neighbors=150)

# Create a Voting Classifier
voting_classifier = VotingClassifier(estimators=[
    ('ann', ann_model),
    ('svm', svm_model),
    ('rf', rf_model),
    ('lr', lr_model),
    ('xgb', xgb_model),
    ('ada', ada_model),
    ('nb', nb_model),
    ('knn',knn_model)
], voting='soft')  # 'soft' uses probability estimates for voting

voting_classifier.fit(X_train_std, y_train)

# Make predictions using the ensemble model
ensemble_predictions = voting_classifier.predict(X_test_std)
ensemble_predictions = np.where(ensemble_predictions > 0.5, 1, 0)

# Evaluate the ensemble's performance
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, ensemble_predictions)
print(f'Ensemble Accuracy: {accuracy:.2f}')

In [ ]:
df_pars_test = pd.read_csv('Testset_features.csv')
pars_test = np.array(df_pars_test)
pars_test

In [ ]:
X_testset = pars_test[:,2:]
sc = StandardScaler()
sc.fit(X_testset)
X_testsub_std = sc.transform(X_testset)

In [ ]:
y_pred_test = voting_classifier.predict(X_testsub_std)
y_pred_test

In [ ]:
Ans = pd.DataFrame(y_pred_test)
#Ans.to_csv('Test_Submission_20000_Ens.csv')

In [ ]:
## leaky relu, accuracy 0.8282 with one hidden layer and 32
## scaled data, leaky relu, acc 0.8308, 2 hidden layers and 64,32
## scaled data, relu, acc 0.831, 2 hidden layers and 64,32
## scaled data, leaky relu, acc 0.8308, 2 hidden layers and 64,32
## scaled data, (LR(i/p) PRelu prelu), acc 0.8348, 2 hidden layers and 128,64
## scaled data, (LR(i/p) PRelu prelu), acc 0.8288, 2 hidden layers and 128,128
## scaled data, (LR(i/p) PRelu prelu), acc 0.8348, 2 hidden layers and 128,64
scaled data, (LR(input) PRelu prelu), acc 0.8358, 2 hidden layers and 128,32
## scaled data, (LR(i/p) PRelu prelu), acc 0.8308, 2 hidden layers and 128,16
## scaled data, (elu(i/p) elu elu), acc 0.8328, 2 hidden layers and 64,32
## scaled data, (elu(i/p) elu elu), acc 0.8326, 2 hidden layers and 128,32
scaled data, (elu(i/p) elu elu), acc 0.8358, 2 hidden layers and 128,64
## scaled data, (elu(i/p) elu elu), acc 0.8304, 2 hidden layers and 128,128
## scaled data, (selu(i/p) selu selu), acc 0.8318, 2 hidden layers and 64,32
## scaled data, (selu(i/p) selu selu), acc 0.829, 2 hidden layers and 128,32
## scaled data, (selu(i/p) selu selu), acc 0.831, 2 hidden layers and 64,64
## scaled data, (selu(i/p) selu selu), acc 0.834, 2 hidden layers and 64,16
## scaled data, (selu(i/p) selu selu), acc 0.829, 2 hidden layers and 64,8
## scaled data, (crelu(i/p) crelu crelu), acc 0.8344, 2 hidden layers and 64,32
## scaled data, (crelu(i/p) crelu crelu), acc 0.8252, 2 hidden layers and 64,64
## scaled data, (crelu(i/p) crelu crelu), acc 0.8342, 2 hidden layers and 64,16
## scaled data, (crelu(i/p) crelu crelu), acc 0.832, 2 hidden layers and 128,64

In [ ]:
ann = tf.keras.Sequential()

# Add the first hidden layer with Leaky ReLU activation
ann.add(tf.keras.layers.Dense(units=10, activation=tf.nn.leaky_relu))

# Add the second hidden layer with PReLU activation
ann.add(tf.keras.layers.Dense(units=128))
ann.add(tf.keras.layers.PReLU())

# Add the third hidden layer with PReLU activation
ann.add(tf.keras.layers.Dense(units=30))
ann.add(tf.keras.layers.PReLU())

# Add the output layer with sigmoid activation
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
import tensorflow as tf

# Create a Sequential model
ann = tf.keras.Sequential()

# Add the first hidden layer with ELU activation
ann.add(tf.keras.layers.Dense(units=10, activation='elu'))

# Add the second hidden layer with ELU activation
ann.add(tf.keras.layers.Dense(units=128, activation='elu'))

# Add the third hidden layer with ELU activation
ann.add(tf.keras.layers.Dense(units=128, activation='elu'))

# Add the output layer with sigmoid activation
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
import tensorflow as tf

# Create a Sequential model
ann = tf.keras.Sequential()

# Add the first hidden layer with SELU activation
ann.add(tf.keras.layers.Dense(units=10, activation='selu'))

# Add the second hidden layer with SELU activation
ann.add(tf.keras.layers.Dense(units=64, activation='selu'))

# Add the third hidden layer with SELU activation
ann.add(tf.keras.layers.Dense(units=8, activation='selu'))

# Add the output layer with sigmoid activation
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Create a custom RReLU activation layer
class RReLU(tf.keras.layers.Layer):
    def __init__(self, lower=0.225, upper=0.433):
        super(RReLU, self).__init__()
        self.lower = lower
        self.upper = upper

    def call(self, inputs):
        alpha = tf.random.uniform(shape=tf.shape(inputs), minval=self.lower, maxval=self.upper)
        return tf.maximum(alpha * inputs, inputs)

# Create a Sequential model
ann = tf.keras.Sequential()

# Add the first hidden layer with RReLU activation
ann.add(tf.keras.layers.Dense(units=10))
ann.add(RReLU())

# Add the second hidden layer with RReLU activation
ann.add(tf.keras.layers.Dense(units=128))
ann.add(RReLU())

# Add the third hidden layer with RReLU activation
ann.add(tf.keras.layers.Dense(units=32))
ann.add(RReLU())

# Add the output layer with sigmoid activation
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Concatenate

# Define a custom CReLU layer
class CReLU(Layer):
    def __init__(self):
        super(CReLU, self).__init__()

    def call(self, inputs):
        positive = tf.keras.activations.relu(inputs)
        negative = tf.keras.activations.relu(-inputs)
        return Concatenate()([positive, negative])

# Create a Sequential model
ann = tf.keras.Sequential()

# Add the first hidden layer with CReLU activation
ann.add(tf.keras.layers.Dense(units=10, activation=CReLU()))

# Add the second hidden layer with CReLU activation
ann.add(tf.keras.layers.Dense(units=64, activation=CReLU()))

# Add the third hidden layer with CReLU activation
ann.add(tf.keras.layers.Dense(units=16, activation=CReLU()))

# Add the output layer with sigmoid activation
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])